<a href="https://colab.research.google.com/github/maryanneg/projeto-despesas-publicas/blob/main/DESPESAS_PUBLICAS_BAIXAR_E_ANALISAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import requests
import os
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets

DIRETORIO_DESTINO = "dados_despesa"

MESES = {
    "JANEIRO": "01", "FEVEREIRO": "02", "MARÇO": "03",
    "ABRIL": "04", "MAIO": "05", "JUNHO": "06",
    "JULHO": "07", "AGOSTO": "08", "SETEMBRO": "09",
    "OUTUBRO": "10", "NOVEMBRO": "11", "DEZEMBRO": "12"
}

valores_ano = ["Todos"] + [str(a) for a in range(2014, 2026)]
valores_mes = ["Todos"] + list(MESES.keys())

combo_ano = widgets.Dropdown(options=valores_ano, description='Ano:')
combo_mes = widgets.Dropdown(options=valores_mes, description='Mês:')
botao_baixar = widgets.Button(description='Baixar e Extrair')
botao_analisar = widgets.Button(description='Analisar Dados')

output_baixar = widgets.Output()
output_analise = widgets.Output()

display(combo_ano, combo_mes, botao_baixar, output_baixar, widgets.HTML("<hr>"),
        botao_analisar, output_analise, widgets.HTML("<hr>"))


def baixar_e_extrair(b):
    ano = combo_ano.value
    mes_nome = combo_mes.value.upper()

    with output_baixar:
        output_baixar.clear_output()
        if not ano or not mes_nome:
            print("Por favor, selecione o ano e o mês.")
            return

        if ano == "Todos":
            anos = [str(a) for a in range(2014, 2026)]
        else:
            anos = [ano]

        if mes_nome == "TODOS":
            meses = list(MESES.values())
        else:
            mes_num = MESES.get(mes_nome)
            if not mes_num:
                print(f"Mês inválido: {mes_nome}")
                return
            meses = [mes_num]

        total = len(anos) * len(meses)
        count = 0

        erros = []
        baixados = []

        os.makedirs(DIRETORIO_DESTINO, exist_ok=True)

        for a in anos:
            for m in meses:
                count += 1
                print(f"Baixando {count} de {total} ({a}/{m})...")

                codigo = f"{a}{m}"
                url = f"https://portaldatransparencia.gov.br/download-de-dados/despesas-execucao/{codigo}"
                nome_zip = f"despesas-execucao-{codigo}.zip"
                caminho_zip = os.path.join(DIRETORIO_DESTINO, nome_zip)

                try:
                    resposta = requests.get(url)
                    resposta.raise_for_status()

                    with open(caminho_zip, "wb") as f:
                        f.write(resposta.content)

                    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
                        zip_ref.extractall(DIRETORIO_DESTINO)

                    os.remove(caminho_zip)
                    baixados.append(codigo)

                except requests.exceptions.RequestException:
                    erros.append(codigo)
                except zipfile.BadZipFile:
                    erros.append(codigo)

        print("\nResultado:")
        if baixados:
            print(f"Downloads concluídos para: {', '.join(baixados)}")
        if erros:
            print(f"Falha ao baixar/extrair: {', '.join(erros)}")
        if not baixados and not erros:
            print("Nenhum arquivo baixado.")
        print("Pronto!")


def analisar_dados_existentes(ano_selecionado, mes_selecionado):
    with output_analise:
        output_analise.clear_output()

        # Filtrar anos
        if ano_selecionado == "Todos":
            anos = [str(a) for a in range(2014, 2026)]
        else:
            anos = [ano_selecionado]

        # Filtrar meses
        if mes_selecionado == "Todos":
            meses = list(MESES.values())
        else:
            meses = [MESES[mes_selecionado]]

        arquivos = []
        for ano in anos:
            for mes in meses:
                padrao = f"{ano}{mes}"
                encontrados = [f for f in os.listdir(DIRETORIO_DESTINO)
                              if f.startswith(padrao) and f.endswith('_Despesas.csv')]
                arquivos.extend(encontrados)

        if not arquivos:
            print("Nenhum arquivo de despesa encontrado para o filtro selecionado.")
            return

        print(f"Analisando {len(arquivos)} arquivo(s) para {ano_selecionado} / {mes_selecionado}...\n")

        lista_df = []
        for arq in arquivos:
            caminho = os.path.join(DIRETORIO_DESTINO, arq)
            try:
                df = pd.read_csv(caminho, sep=';', encoding='latin1')

                col_valor = 'Valor Pago (R$)'
                if col_valor not in df.columns:
                    print(f"Coluna '{col_valor}' não encontrada em {arq}. Pulando arquivo.")
                    continue

                df[col_valor] = df[col_valor].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
                df[col_valor] = pd.to_numeric(df[col_valor], errors='coerce').fillna(0)

                ano = int(arq[:4])
                mes = int(arq[4:6])
                df['Ano'] = ano
                df['Mês'] = mes

                lista_df.append(df)

            except Exception as e:
                print(f"Erro ao ler {arq}: {e}")

        if not lista_df:
            print(" dado válido para análise.")
            return

        df_total = pd.concat(lista_df, ignore_index=True)
        col_valor = 'Valor Pago (R$)'

        total = df_total[col_valor].sum()
        print(f"Total de despesas analisadas: R$ {total:,.2f}\n")

        if 'Nome Órgão Superior' in df_total.columns:
            print("Top 10 órgãos superiores que mais gastaram:")
            gastos_orgao = df_total.groupby('Nome Órgão Superior')[col_valor].sum().sort_values(ascending=False).head(10)

            for orgao, valor in gastos_orgao.items():
                print(f" - {orgao}: R$ {valor:,.2f}")
            print()

            gastos_orgao.plot(kind='bar', figsize=(12,5), title='Top 10 Órgãos Superiores com Maior Gasto')
            plt.ylabel("Valor Pago (R$)")
            plt.xticks(rotation=75)
            plt.tight_layout()
            plt.grid()
            plt.show()
        else:
            print("Coluna 'Nome Órgão Superior' não encontrada para análise.")

        print("Total de despesas por ano:")
        gastos_ano = df_total.groupby('Ano')[col_valor].sum().sort_index()
        for ano, valor in gastos_ano.items():
            print(f" - {ano}: R$ {valor:,.2f}")
        print()

        gastos_ano.plot(kind='bar', figsize=(10,4), title='Despesa Total por Ano')
        plt.ylabel("Valor Pago (R$)")
        plt.tight_layout()
        plt.grid()
        plt.show()



botao_baixar.on_click(baixar_e_extrair)

def ao_clicar_analisar(b):
    ano = combo_ano.value
    mes = combo_mes.value
    analisar_dados_existentes(ano, mes)

botao_analisar.on_click(ao_clicar_analisar)




Dropdown(description='Ano:', options=('Todos', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',…

Dropdown(description='Mês:', options=('Todos', 'JANEIRO', 'FEVEREIRO', 'MARÇO', 'ABRIL', 'MAIO', 'JUNHO', 'JUL…

Button(description='Baixar e Extrair', style=ButtonStyle())

Output()

HTML(value='<hr>')

Button(description='Analisar Dados', style=ButtonStyle())

Output()

HTML(value='<hr>')